### Configuration
#### Load dependencies

In [1]:
import cv2
import os
import face_recognition
from PIL import Image
import random
import numpy as np
import matplotlib.pyplot as plt
import uuid

# import tf dependencies
import tensorflow as tf
from keras.models import Model
from keras.layers import Layer, Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Flatten, BatchNormalization, Dropout, Dense, LeakyReLU



2023-07-22 00:39:11.648619: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-22 00:39:11.692039: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-22 00:39:11.692736: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-22 00:39:12.302801: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


#### Config GPU

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
    print(gpu)

#### Define constants

In [3]:
POSITIVE_PATH = os.path.normpath(os.path.join('data', 'positive'))
NEGATIVE_PATH = os.path.normpath(os.path.join('data', 'negative'))
ANCHOR_PATH = os.path.normpath(os.path.join('data', 'anchor'))
PIXELS_SIZE = 100

#### Create directories

In [4]:
os.makedirs(POSITIVE_PATH, exist_ok=True)
os.makedirs(NEGATIVE_PATH, exist_ok=True)
os.makedirs(ANCHOR_PATH, exist_ok=True)


### Data collection

In [5]:
cap = cv2.VideoCapture(0)
face_locations = None
capture_mode = None

while cap.isOpened():
    ret, frame = cap.read()
    if face_locations is None:
        face_locations = face_recognition.face_locations(frame)
        t, r, b, l = face_locations[0]
        y_margin = 250 - abs(t-b)
        x_margin = 250 - abs(l-r)
     # get 250 x 250 image basing on initial face position
    frame = frame[t-y_margin//2: b+y_margin//2, l - x_margin//2:r+x_margin//2, :]

    key = cv2.waitKey(1) & 0XFF
    # Change mode
    if key == ord('a'):
        capture_mode = 'a'
    elif key == ord('p'):
        capture_mode = 'p'
    elif key == ord('q'):
        break

    # collect samples based on current mode
    if capture_mode == 'a':
        cv2.imwrite(os.path.join(ANCHOR_PATH, f'{uuid.uuid1()}.jpg'), frame)
    elif capture_mode == 'p':
        cv2.imwrite(os.path.join(POSITIVE_PATH, f'{uuid.uuid1()}.jpg'), frame)

    cv2.imshow('Image collection', frame)

cap.release()
cv2.destroyAllWindows()

### Image preprocessing

In [6]:
anchor = tf.data.Dataset.list_files(os.path.join(ANCHOR_PATH, '*.jpg'))
positive = tf.data.Dataset.list_files(os.path.join(POSITIVE_PATH, '*.jpg'))
negative = tf.data.Dataset.list_files(os.path.join(NEGATIVE_PATH, '*.jpg'))

#### Define functions

In [7]:
def preprocess(path_to_file):
    """Function that read file from file, resize and rescale numpy array"""
    img = tf.io.decode_jpeg(tf.io.read_file(path_to_file))
    return tf.image.resize(img, (PIXELS_SIZE, PIXELS_SIZE)) / 255

def preprocess_twin(input_img, val_img, label):
    return preprocess(input_img), preprocess(val_img), label

#### Define dataset

In [8]:
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negatives)

In [9]:
data = data.map(preprocess_twin)
data = data.cache()
data.shuffle(buffer_size=1024)

<_ShuffleDataset element_spec=(TensorSpec(shape=(100, 100, None), dtype=tf.float32, name=None), TensorSpec(shape=(100, 100, None), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.float32, name=None))>

In [10]:
TRAIN_RATIO = 0.7
TEST_RATIO = 1 - TRAIN_RATIO

In [11]:
test_data = data.skip(round(len(data)*TRAIN_RATIO))
test_data = test_data.take(round(len(data)*TEST_RATIO))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)


### Define the model
#### Create embedding layers

In [12]:
def make_embedding():
    inp = Input(shape=(PIXELS_SIZE,PIXELS_SIZE,3), name='input_image')
    leaky_relu = LeakyReLU()

    # First block
    c1 = Conv2D(64, (10,10))(inp)
    c1 = leaky_relu(c1)
    c1 = BatchNormalization()(c1)
    m1 = MaxPooling2D((2,2), padding='same')(c1)

    # Second block
    c2 = Conv2D(128, (7,7))(m1)
    c2 = leaky_relu(c2)
    c2 = BatchNormalization()(c2)
    m2 = MaxPooling2D((2,2), padding='same')(c2)

    # Third block
    c3 = Conv2D(128, (4,4))(m2)
    c3 = leaky_relu(c3)
    c3 = BatchNormalization()(c3)
    m3 = MaxPooling2D((2,2), padding='same')(c3)

    # Final embedding block
    c4 = Conv2D(256, (4,4))(m3)
    c4 = leaky_relu(c4)
    c4 = BatchNormalization()(c4)


    f1 = Flatten()(c4)
    d1 = Dense(1024, activation='relu')(f1)
    d1 = Dropout(0.5)(d1)
    d2 = Dense(4096, activation='sigmoid')(d1)

    return Model(inputs=[inp], outputs=[d2], name='embedding')

In [13]:
embedding = make_embedding()
embedding.summary()

Model: "embedding"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_image (InputLayer)       [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 91, 91, 64)   19264       ['input_image[0][0]']            
                                                                                                  
 leaky_re_lu (LeakyReLU)        multiple             0           ['conv2d[0][0]',                 
                                                                  'conv2d_1[0][0]',               
                                                                  'conv2d_2[0][0]',       

#### A distance layer - Siamese Distance class

In [14]:
class L1Dist(Layer):
    def __init__(self, **kwargs):
        super().__init__()

    def call(self, input_emb, val_emb):
        return tf.math.abs(input_emb-val_emb)

#### Create Siamese model

In [21]:
def make_siamese_model(num_classes=None):
    """
    Create a siamese network model.
    This network will have two inputs (anchor, validation),
    each one goes through the same embedding model and the output embeddings are compared.
    """
    # Anchor image input in the network
    input_img = Input(shape=(PIXELS_SIZE,PIXELS_SIZE,3), name='anchor_input')

    # Validation image in the network
    val_img = Input(shape=(PIXELS_SIZE,PIXELS_SIZE,3), name='validation_input')
    # Combine siamese distance components
    siam_layer =  L1Dist(name='distance')
    distances =siam_layer(embedding(input_img), embedding(val_img))

    if num_classes == None:
        classifier = Dense(1, activation='sigmoid')(distances)
    else:
        output = Dense(num_classes, activation='softmax')(distances)


    return Model(inputs=[input_img, val_img], outputs=classifier, name='SiameseNetwork')


In [22]:
clf = make_siamese_model()

### Training